
    Use SQL to create a schema with 4 tables: stores, store_states, train, and test in sqlite.
    Inner join stores and store_states to add state information to the store information. Call the resulting table stores_with_state(look up the syntax for saving the results of a join to a table). Why does this make sense as an inner join?
    Left join train and test tables individually with the stores_with_state table to create train_stores and test_stores. Why does this join need to be a left join from the perspective of predicting sales on the test set?
    Transfer these two tables to duckdb. Save duckdb database files as well as parquet files
    Compute grouby statistics on sales amounts on store-ids, day of week, and their combination.Save this last analytics as a table in the database and as a parquet file. You can do these calculations either on the database files or the parquet files.


In [ ]:
!wget https://github.com/rahuldave/SQLForDataScientists/raw/main/data/train_cleaned.csv.gz
!wget https://github.com/rahuldave/SQLForDataScientists/raw/main/data/test_cleaned.csv
!wget https://github.com/rahuldave/SQLForDataScientists/raw/main/data/store_cleaned.csv
!wget https://github.com/rahuldave/SQLForDataScientists/raw/main/data/store_states.csv

--2023-04-23 15:15:46--  https://github.com/rahuldave/SQLForDataScientists/raw/main/data/train_cleaned.csv.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rahuldave/SQLForDataScientists/main/data/train_cleaned.csv.gz [following]
--2023-04-23 15:15:46--  https://raw.githubusercontent.com/rahuldave/SQLForDataScientists/main/data/train_cleaned.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7621669 (7.3M) [application/octet-stream]
Saving to: ‘train_cleaned.csv.gz’

train_cleaned.csv.g 100%[===================>]   7.27M  --.-KB/s    in 0.08s   

2023-04-23 15:15:46 (86.1 MB/s) - ‘train_cl

In [ ]:
!gunzip train_cleaned.csv.gz

In [ ]:
!ls

sample_data	   store_states.csv  train_cleaned.csv
store_cleaned.csv  test_cleaned.csv


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("test_cleaned.csv")
df.head(5)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,True,True,False,False
1,2,3,4,2015-09-17,True,True,False,False
2,3,7,4,2015-09-17,True,True,False,False
3,4,8,4,2015-09-17,True,True,False,False
4,5,9,4,2015-09-17,True,True,False,False


In [ ]:
import sqlite3 as sql

In [ ]:
## create connection

database="sales.db"
conn=sql.connect(database)
cursor = conn.cursor()



In [ ]:
df.to_sql("test",conn)

41088

In [ ]:
query="select * from test"

In [ ]:
df=pd.read_sql_query(query, conn)


In [ ]:
df.head(5)

,index,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,0,1,1,4,2015-09-17,1,1,0,0
1,1,2,3,4,2015-09-17,1,1,0,0
2,2,3,7,4,2015-09-17,1,1,0,0
3,3,4,8,4,2015-09-17,1,1,0,0
4,4,5,9,4,2015-09-17,1,1,0,0


In [ ]:
query="""CREATE TABLE stores_with_state AS
SELECT stores.*, store_states.state
FROM stores
INNER JOIN store_states
ON stores.Store = store_states.Store;
"""

In [ ]:
cursor.execute(query)

In [ ]:
query2="select * from stores_with_state"

In [ ]:
df=pd.read_sql_query(query2, conn)

In [ ]:
df.columns

Index(['index', 'Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State'],
      dtype='object')

In [ ]:
query="""CREATE TABLE train_stores AS
SELECT train.*, stores_with_state.*
FROM train
LEFT JOIN stores_with_state
ON train.Store = stores_with_state.Store"""


In [ ]:
cursor.execute(query)
query2="select * from train_stores"

In [ ]:
df=pd.read_sql_query(query2, conn)
df.head(5)

,index,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,0,1,5,2015-07-31,5263,555,1,1,0,1,...,c,a,1270.0,9,2008,0,1,1900,None,HE
1,1,2,5,2015-07-31,6064,625,1,1,0,1,...,a,a,570.0,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",TH
2,2,3,5,2015-07-31,8314,821,1,1,0,1,...,a,a,14130.0,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",NW
3,3,4,5,2015-07-31,13995,1498,1,1,0,1,...,c,c,620.0,9,2009,0,1,1900,None,BE
4,4,5,5,2015-07-31,4822,559,1,1,0,1,...,a,a,29910.0,4,2015,0,1,1900,None,SN


In [ ]:
df.shape

(1017209, 22)

In [ ]:
df.isnull().any()

index                        False
Store                        False
DayOfWeek                    False
Date                         False
Sales                        False
Customers                    False
Open                         False
Promo                        False
StateHoliday                 False
SchoolHoliday                False
index:1                      False
Store:1                      False
StoreType                    False
Assortment                   False
CompetitionDistance           True
CompetitionOpenSinceMonth    False
CompetitionOpenSinceYear     False
Promo2                       False
Promo2SinceWeek              False
Promo2SinceYear              False
PromoInterval                 True
State                        False
dtype: bool

In [ ]:
import pandas as pd
import duckdb

# Connect to the SQLite database
conn = sql.connect('sales.db')

# Read the "train_stores" and "test_stores" tables into Pandas dataframes
train_stores_df = pd.read_sql_query("SELECT * FROM train_stores", conn)
stores_with_state_df = pd.read_sql_query("SELECT * FROM stores_with_state", conn)

# Connect to DuckDB and create the database
duckdb_conn = duckdb.connect('sales.duckdb')

# Insert the dataframes into DuckDB tables
duckdb_conn.register("train_stores", train_stores_df)
duckdb_conn.register("stores_with_state", stores_with_state_df)

# Save the dataframes as Parquet files
train_stores_df.to_parquet("train_stores.parquet")
stores_with_state_df.to_parquet("test_stores.parquet")

# Close the database connections
conn.close()
duckdb_conn.close()
